# In this file data about groups, events and RSVPs are gathered, filtered and transferred to CSV files.

# 1. Preparation: required libraries are fetched and connection to meetup is established.

In [2]:
    !pip install meetup.api
    import meetup.api
    import json

    import pandas
    from pandas.io.json import json_normalize

    
    client = meetup.api.Client('149155b67979442634b463d11cc')
    

  Using cached https://files.pythonhosted.org/packages/b7/44/a545b860f19cac088cb7f3e39beae8334b63e025729348ddeffb3342122a/meetup_api-0.1.1-py2.py3-none-any.whl


# 2. function createCSV is prepared

In [3]:
    # creation of the CSV files for Groups, Events and RSVPs
    def createCSV(eventsRange, nameOfEventsRange):
        fileGroups = "groups" + nameOfEventsRange + ".csv"
        fileRSVPs = "rsvps" + nameOfEventsRange + ".csv"
        fileEvents = "events" + nameOfEventsRange + ".csv"
        
        groups =""
        f = open(fileRSVPs, "w+")
        f.close()

        f = open(fileEvents, "w+")
        f.close()
        
        #create Events-CSV
        eventDF = pandas.DataFrame.from_dict(eventsRange.__dict__["results"])
        eventDF = eventDF[['id', 'name', 'event_url', 'rsvp_limit','yes_rsvp_count', 'time', 'venue','group']]
    
        eventDF = eventDF.to_json(orient='records')
        # flatten Dateframe
        eventDF = json_normalize(json.loads(eventDF))[['id', 'name', 'event_url', 'rsvp_limit','yes_rsvp_count', 'time','venue.city','venue.address_1','group.id', 'group.urlname']].rename(columns={'venue.address_1':'address','venue.city': 'city','group.id': 'group_id', 'group.urlname': 'group_urlname'}) 
        
        
        eventDF.to_csv(fileEvents, mode='a', encoding='utf-8')

        for index2, row2 in eventDF.iterrows():
            eventId = row2['id']
            groups += row2['group_urlname']+","

            #gather RSVP-data
            rsvp_info = client.GetRsvps({'event_id': eventId})
            rsvpDF = pandas.DataFrame.from_dict(rsvp_info.__dict__["results"])
            #create RSVP-CSV
            rsvpDF.loc[:, ('member')]
            #rsvpDF = rsvpDF[['member']]
            rsvpDF = rsvpDF.to_json(orient='records')
            # flatten Dataframe
            rsvpDF = json_normalize(json.loads(rsvpDF))[['member.name', 'member.member_id']].rename(columns={'member.name': 'name', 'member.member_id': 'member_id'}) 
            rsvpDF['event_id'] = eventId
            rsvpDF.to_csv(fileRSVPs, mode='a', encoding='utf-8')

        #remove last comma
        groups = groups[:-1]
        #gather Group-data
        groupRange = client.GetGroups({'group_urlname' : groups})
        groupDF = pandas.DataFrame.from_dict(groupRange.__dict__["results"])
        #create Group-CSV
        groupDF = groupDF[['id', 'name', 'members', 'link', 'topics']]  
        groupDF.to_csv(fileGroups, encoding='utf-8')
        

# 3. The events are the basis for the data collection. Via an API, data about the events are fetched in the desired range.
Depending on the desired analysis, the corresponding line is used. The other lines are commented out. 

In [5]:
%%capture
#ZH = client.GetOpenEvents(category=34, city="Zürich", country='CH')
# info: 28264920 -> code4fun ---- 31154587 --> swiss data circle
Oracle = client.GetEvents({'status': 'past', 'group_id' : '28264920,31154587'})

#BS = client.GetOpenEvents(category=34, city="Basel", country='CH')
#Lndn = client.GetOpenEvents(category=34, city="London", country='GB')
#RHF = client.GetOpenEvents(category=34, city="Rheinfelden", country='CH')

#Top = client.GetOpenEvents({'category': 34, 'country' : 'CH', 'time' : '1m,12m', 'status' : 'upcoming', 'trending' : 'desc=true'})
#ZHTrend = client.GetOpenEvents(category=34, country = "CH", city="Zürich", time="0,12m", status="upcoming", radius=150)

# 4. The function for CSV creation is called here using Events-Dataframe. The names of the files to be created are specified in the parameters.
Depending on the desired analysis, the corresponding line is used. The other lines are commented out. 

In [6]:
%%capture
#createCSV(ZH, "ZH")

createCSV(Oracle,"Oracle")

#createCSV(Top, "Top")
#createCSV(ZHTrend, "ZHTrend")
